### Hyperparameter Tuning

In [ ]:
from resource_credentials import (subscription_key, 
    resource_group_name, 
    workspace_name)

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Getting client
ml_client = MLClient(credential=DefaultAzureCredential(), 
                     subscription_id=subscription_key, 
                     resource_group_name=resource_group_name,
                     workspace_name=workspace_name)

In [ ]:
from azure.ai.ml import command

# create job
job = command(
    code="./src",
    command="python train-classification-model.py --training-data diabetes.csv --reg_rate ${{inputs.reg_rate}}",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="compute-instance-sdk",
    experiment_name="diabetes-training-tunning",
    inputs={
        "reg_rate": 0.01,
    },
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

In [ ]:
# Define the search space for hyperparameter tuning
from azure.ai.ml.sweep import Choice

# Discrete hyperparameter space
command_job = job(reg_rate=Choice([0.01, 0.1, 1.0]))

sweep_job = command_job.sweep(
    compute="compute-instance-sdk",
    sampling_algorithm="grid",
    primary_metric="Accuracy",
    goal="maximize")

sweep_job.experiment_name = "diabetes-training-tunning"

sweep_job.set_limits(max_total_trials=4, max_concurrent_trials=2, timeout=7200)

# submit job
returned_job = ml_client.create_or_update(sweep_job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)
